# Getting Started With 3D Data in Scale Nucleus

---



In this tutorial, we'll walk through a demo of how to upload 3D Scenes to Nucleus using the open source Pandaset dataset

### Step 1: Install dependencies

In [5]:
%%bash
pip install scale-nucleus
pip install boto3
pip install botocore

You should consider upgrading via the '/Users/sashaharrison/Library/Caches/pypoetry/virtualenvs/scale-nucleus-KNrjfYd0-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sashaharrison/Library/Caches/pypoetry/virtualenvs/scale-nucleus-KNrjfYd0-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sashaharrison/Library/Caches/pypoetry/virtualenvs/scale-nucleus-KNrjfYd0-py3.8/bin/python -m pip install --upgrade pip' command.


### Step 2: Read PandaSet files from S3

More information about pandaset can be found at: https://scale.com/open-datasets/pandaset

For the purposes of this demo, we've pre-processed the Pandaset dataset into a more convenient, and made it accessible via a public S3 bucket. Pandaset is licensed under the following terms of use: https://scale.com/legal/pandaset-terms-of-use

In [6]:
import os
import re
import json
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import nucleus
from nucleus import NucleusClient, DatasetItem, Frame, LidarScene

PUBLIC_PANDASET_BUCKET = "pandaset-public"

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
paginator = s3.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket=PUBLIC_PANDASET_BUCKET)

In [7]:
object_paths = []
for page in result:
    if "Contents" in page:
        for key in page["Contents"]:
            object_path = key["Key"]
            object_paths.append(object_path)

In [8]:
print(object_paths)

['pandaset_0/.DS_Store', 'pandaset_0/001/.DS_Store', 'pandaset_0/001/LICENSE.txt', 'pandaset_0/001/annotations/.DS_Store', 'pandaset_0/001/annotations/cuboids/00.json', 'pandaset_0/001/annotations/cuboids/01.json', 'pandaset_0/001/annotations/cuboids/02.json', 'pandaset_0/001/annotations/cuboids/03.json', 'pandaset_0/001/annotations/cuboids/04.json', 'pandaset_0/001/annotations/cuboids/05.json', 'pandaset_0/001/annotations/cuboids/06.json', 'pandaset_0/001/annotations/cuboids/07.json', 'pandaset_0/001/annotations/cuboids/08.json', 'pandaset_0/001/annotations/cuboids/09.json', 'pandaset_0/001/annotations/cuboids/10.json', 'pandaset_0/001/annotations/cuboids/11.json', 'pandaset_0/001/annotations/cuboids/12.json', 'pandaset_0/001/annotations/cuboids/13.json', 'pandaset_0/001/annotations/cuboids/14.json', 'pandaset_0/001/annotations/cuboids/15.json', 'pandaset_0/001/annotations/cuboids/16.json', 'pandaset_0/001/annotations/cuboids/17.json', 'pandaset_0/001/annotations/cuboids/18.json', 'pa

# Data Exploration

Here's how our dataset is structured - <TODO add some exposition here>
    
```
001/
  lidar/
    00.json
    01.json
        ...
    79.json
  camera/
    back_camera/
        00.jpg
        01.jpg
        ...
        79.jpg
026/
  lidar/
    00.json
    01.json
        ...
    79.json
  camera/
    back_camera/
        00.jpg
        01.jpg
        ...
        79.jpg
```

In [15]:
# Define some helper functions
def is_image_path(object_path):
    return "camera/" in object_path and ".jpg" in object_path

def is_pointcloud_path(object_path):
    return "lidar/" in object_path and ".json" in object_path
  
def is_cuboid_path(object_path):
    return "cuboids/" in object_path and ".json" in object_path

def read_json(path):
    s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
    content_object = s3.Object(BUCKET, path)
    file_content = content_object.get()['Body'].read().decode('utf-8')
    return json.loads(file_content)

In [16]:
image_paths = []
pointcloud_paths = []
cuboid_paths = []

for path in object_paths:
    if is_image_path(path):
        image_paths.append(path)
    elif is_pointcloud_path(path):
        pointcloud_paths.append(path)
    elif is_cuboid_path(path):
        cuboid_paths.append(path)

In [17]:
print(image_paths)
print(pointcloud_paths)
print(cuboid_paths)

['pandaset_0/001/camera/back_camera/00.jpg', 'pandaset_0/001/camera/back_camera/01.jpg', 'pandaset_0/001/camera/back_camera/02.jpg', 'pandaset_0/001/camera/back_camera/03.jpg', 'pandaset_0/001/camera/back_camera/04.jpg', 'pandaset_0/001/camera/back_camera/05.jpg', 'pandaset_0/001/camera/back_camera/06.jpg', 'pandaset_0/001/camera/back_camera/07.jpg', 'pandaset_0/001/camera/back_camera/08.jpg', 'pandaset_0/001/camera/back_camera/09.jpg', 'pandaset_0/001/camera/back_camera/10.jpg', 'pandaset_0/001/camera/back_camera/11.jpg', 'pandaset_0/001/camera/back_camera/12.jpg', 'pandaset_0/001/camera/back_camera/13.jpg', 'pandaset_0/001/camera/back_camera/14.jpg', 'pandaset_0/001/camera/back_camera/15.jpg', 'pandaset_0/001/camera/back_camera/16.jpg', 'pandaset_0/001/camera/back_camera/17.jpg', 'pandaset_0/001/camera/back_camera/18.jpg', 'pandaset_0/001/camera/back_camera/19.jpg', 'pandaset_0/001/camera/back_camera/20.jpg', 'pandaset_0/001/camera/back_camera/21.jpg', 'pandaset_0/001/camera/back_cam

### Step 3: Construct LidarScenes

In [18]:
BUCKET = "pandaset-public"
S3_BUCKET = "s3://pandaset-public"

In [41]:
# For this demo, we will upload scenes 001, 006, and 023 from PandaSet
SCENE_IDS = ["001", "006", "023"]
CAMERA_SENSORS = ["back_camera", "front_camera", "front_left_camera", "front_right_camera", "left_camera", "right_camera"]

def construct_nucleus_scenes():
    scenes = []
    for scene_id in SCENE_IDS:
        scene_ref_id = f"scene-{scene_id}"
        scene = LidarScene(scene_ref_id)

        camera_sensor_to_params = {}
        for sensor in CAMERA_SENSORS:
            base_path = f"pandaset_0/{scene_id}/camera/{sensor}/"
            intrinsics_path = os.path.join(base_path, 'intrinsics.json')
            poses_path = os.path.join(base_path, 'poses.json')
            intrinsics = read_json(intrinsics_path)
            poses = read_json(poses_path)
            camera_params = {"intrinsics": intrinsics, "poses": poses}
            camera_sensor_to_params[sensor] = camera_params

        image_paths_in_scene = [path for path in image_paths if f"{scene_id}/" in path]
        for image_path in image_paths_in_scene:
            tokens = re.split('/|\.', image_path)
            frame_idx = int(tokens[-2])
            sensor_name = tokens[-3]

            params = camera_sensor_to_params[sensor_name]
            pose = params["poses"][frame_idx]
            camera_params = {**params["intrinsics"], **pose}

            image_url = os.path.join(S3_BUCKET, image_path)    
            reference_id = f"scene-{scene_id}-frame-{frame_idx}-{sensor_name}"
            metadata = {"camera_params": camera_params}
            item = DatasetItem(image_location=image_url, reference_id=reference_id, metadata=metadata)
            scene.add_item(frame_idx, sensor_name, item)

        pointcloud_paths_in_scene = [path for path in pointcloud_paths if f"{scene_id}/" in path]
        for pointcloud_path in pointcloud_paths_in_scene:
            tokens = re.split('/|\.', pointcloud_path)
            frame_idx = int(tokens[-2])
            sensor_name = tokens[-3]

            pointcloud_url = os.path.join(S3_BUCKET, pointcloud_path)
            reference_id = f"scene-{scene_id}-frame-{frame_idx}-{sensor_name}"
            item = DatasetItem(pointcloud_location=pointcloud_url, reference_id=reference_id)
            scene.add_item(frame_idx, sensor_name, item)

        scenes.append(scene)
    return scenes

In [42]:
scenes = construct_nucleus_scenes()
scene_1 = scenes[0]
print("number of lidar DatasetItems:", len(scene_1.get_items_from_sensor("lidar")))
print("number of DatasetItems:", len(scene_1.get_items()))
print("number of frames:", scene_1.length)
print("number of sensors:", scene_1.num_sensors)
print("sensors:", scene_1.get_sensors())

number of lidar DatasetItems: 11
number of DatasetItems: 77
number of frames: 11
number of sensors: 7
sensors: ['back_camera', 'left_camera', 'front_left_camera', 'right_camera', 'front_right_camera', 'front_camera', 'lidar']


### Step 4: Append Scenes to Dataset

In [47]:
API_KEY = "live_318209d04e3746dbafbe1f195a4a1872"
TEST_DATASET_NAME = "pandaset_3d"

In [48]:
client = NucleusClient(API_KEY)
dataset = client.create_dataset(TEST_DATASET_NAME)

In [51]:
append_job = dataset.append(scenes, asynchronous=True)
print(append_job)

AsyncJob(job_id='job_c4dkzmp81a5007g8t6r0', job_last_known_status='Running', job_type='uploadLidarScene', job_creation_time='2021-08-17T04:49:22.405Z', client=NucleusClient(api_key='live_318209d04e3746dbafbe1f195a4a1872', use_notebook=False, endpoint='https://api.scale.com/v1/nucleus'))


In [52]:
append_job.sleep_until_complete()
print(append_job.status())

Status at Mon Aug 16 21:49:27 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:32 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:38 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:43 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:48 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:53 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'Na

In [18]:
print(append_job.status())

{'job_id': 'job_c4deq4cajw800d14p68g', 'status': 'Completed', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}


In [37]:
[print(item.reference_id) for item in scenes[2].get_items()]

scene-023-frame-0-back_camera
scene-023-frame-0-front_camera
scene-023-frame-0-front_left_camera
scene-023-frame-0-front_right_camera
scene-023-frame-0-left_camera
scene-023-frame-0-right_camera
scene-023-frame-0-lidar
scene-023-frame-1-back_camera
scene-023-frame-1-front_camera
scene-023-frame-1-front_left_camera
scene-023-frame-1-front_right_camera
scene-023-frame-1-left_camera
scene-023-frame-1-right_camera
scene-023-frame-1-lidar
scene-023-frame-2-back_camera
scene-023-frame-2-front_camera
scene-023-frame-2-front_left_camera
scene-023-frame-2-front_right_camera
scene-023-frame-2-left_camera
scene-023-frame-2-right_camera
scene-023-frame-2-lidar
scene-023-frame-3-back_camera
scene-023-frame-3-front_camera
scene-023-frame-3-front_left_camera
scene-023-frame-3-front_right_camera
scene-023-frame-3-left_camera
scene-023-frame-3-right_camera
scene-023-frame-3-lidar
scene-023-frame-4-back_camera
scene-023-frame-4-front_camera
scene-023-frame-4-front_left_camera
scene-023-frame-4-front_rig

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Step 5: Upload Annotations

In [53]:
# Let's upload annotations for scene 23
SCENE_ID = "023"
from nucleus import CuboidAnnotation

annotations = []
cuboid_paths_in_scene = [path for path in cuboid_paths if f"{SCENE_ID}/" in path]
for (i, cuboid_path) in enumerate(cuboid_paths_in_scene):
    tokens = re.split('/|\.', cuboid_path)
    frame_idx = int(tokens[-2])
    reference_id = f"scene-{SCENE_ID}-frame-{frame_idx}-lidar"
    
    cuboids_json = read_json(cuboid_path)
    for cuboid in cuboids_json:
        cuboid["reference_id"] = reference_id
    
    new_annotations = [CuboidAnnotation.from_json(cuboid) for cuboid in cuboids_json]
    annotations.extend(new_annotations)

In [54]:
print(len(cuboid_paths_in_scene))
print(len(annotations))

ann = annotations[0]
print(ann)
print(ann.to_payload())

80
12637
CuboidAnnotation(label='Car', position=Point3D(x=-22.581, y=-35.652, z=0.776), dimensions=Point3D(x=1.935, y=4.858, z=1.851), yaw=-0.9474790371, reference_id='scene-023-frame-0-lidar', item_id=None, annotation_id='a494348f-26f4-4559-a0ae-132d4d753159', metadata={'stationary': True, 'camera_used': 5, 'attributes.object_motion': 'Parked', 'cuboids.sibling_id': '-', 'cuboids.sensor_id': -1, 'attributes.pedestrian_behavior': None, 'attributes.pedestrian_age': None, 'attributes.rider_status': None})
{'label': 'Car', 'type': 'cuboid', 'geometry': {'position': {'x': -22.581, 'y': -35.652, 'z': 0.776}, 'dimensions': {'x': 1.935, 'y': 4.858, 'z': 1.851}, 'yaw': -0.9474790371}, 'reference_id': 'scene-023-frame-0-lidar', 'annotation_id': 'a494348f-26f4-4559-a0ae-132d4d753159', 'metadata': {'stationary': True, 'camera_used': 5, 'attributes.object_motion': 'Parked', 'cuboids.sibling_id': '-', 'cuboids.sensor_id': -1, 'attributes.pedestrian_behavior': None, 'attributes.pedestrian_age': None

In [55]:
client.endpoint = 'http://localhost:3000/v1/nucleus'
response = dataset.annotate(annotations)
print(response)

100%|██████████| 3/3 [00:12<00:00,  4.09s/it]

{'dataset_id': 'ds_c4dkzky5vjbg0d1m6fpg', 'annotations_processed': 1685, 'annotations_ignored': 0}


In [22]:
annotate_job.sleep_until_complete()
print(annotate_job.status())

Status at Mon Aug 16 17:57:26 2021: {'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 17:57:31 2021: {'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Completed', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
{'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Completed', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}


In [24]:
item = dataset.iloc(0)
print(item)

{'item': DatasetItem(image_location='s3://pandaset-public/pandaset_0/023/lidar/00.json', reference_id='scene-023-frame-0-lidar', item_id='di_c4der4gkqv9g1xq601q0', metadata={}, pointcloud_location=None), 'annotations': {}}


### TODO: Remove this section
### Step 6: Upload Predictions

In [ ]:
# We'll demonstrate how to upload model predictions using fake predictions
from nucleus import Point3D, CuboidPrediction

def add_noise(obs):
    noise = np.random.normal(0, 1)
    return obs + noise

def add_noise_point(point: Point3D):
    return Point3D(
        add_noise(point.position.x),
        add_noise(point.position.y),
        add_noise(point.position.z)
    )

def add_noise_gt(gt: CuboidAnnotation):
    return CuboidPrediction(
        label=gt.label,
        position=add_noise_point(gt.position),
        dimensions=add_noise_point(gt.dimensions),
        yaw=add_noise(yaw),
        reference_id=gt.reference_id,
        item_id=gt.item_id,
        annotation_id=gt.annotation_id,
        metadata=gt.metadata
    )

predictions = [add_noise_gt(annotation) for annotation in annotations]

In [ ]:
model = client.add_model(name="Test Model", reference_id="test")
model_run = model.create_run(
    name="Test Model Run",
    dataset=dataset,
    predictions=[],
)

predict_job = model_run.predict(predictions, asynchronous=True)
model_run.commit()